In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


In [2]:
import numpy as np
import pandas as pd
train=pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
sample=pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')
test=pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')

In [3]:
print(test.shape)
print(train.shape)
train.head(3)

(250000, 8)
(750000, 9)


,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0


In [4]:
test.head(3)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4


In [5]:
print(train.isnull().sum())
print(train.describe())
print(train.info())

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64
                  id            Age         Height         Weight  \
count  750000.000000  750000.000000  750000.000000  750000.000000   
mean   374999.500000      41.420404     174.697685      75.145668   
std    216506.495284      15.175049      12.824496      13.982704   
min         0.000000      20.000000     126.000000      36.000000   
25%    187499.750000      28.000000     164.000000      63.000000   
50%    374999.500000      40.000000     174.000000      74.000000   
75%    562499.250000      52.000000     185.000000      87.000000   
max    749999.000000      79.000000     222.000000     132.000000   

            Duration     Heart_Rate      Body_Temp       Calories  
count  750000.000000  750000.000000  750000.000000  750000.000000  
mean       15.421015      95.483995      40.036253      88.282781  
std         8.354

In [6]:
print(test.isnull().sum())
print(test.describe())
print(test.info())

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
dtype: int64
                  id            Age         Height         Weight  \
count  250000.000000  250000.000000  250000.000000  250000.000000   
mean   874999.500000      41.452464     174.725624      75.147712   
std     72168.927986      15.177769      12.822039      13.979513   
min    750000.000000      20.000000     127.000000      39.000000   
25%    812499.750000      28.000000     164.000000      63.000000   
50%    874999.500000      40.000000     174.000000      74.000000   
75%    937499.250000      52.000000     185.000000      87.000000   
max    999999.000000      79.000000     219.000000     126.000000   

            Duration     Heart_Rate      Body_Temp  
count  250000.000000  250000.000000  250000.000000  
mean       15.415428      95.479084      40.036093  
std         8.349133       9.450161       0.778448  
min         1.000000     

In [7]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
le=LabelEncoder()
sc=StandardScaler()

In [8]:
train['Sex']=le.fit_transform(train['Sex'])
test['Sex']=le.transform(test['Sex'])

In [9]:
features = ['Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
train['Calories_log'] = np.log1p(train['Calories'])
X = train[features]
y = train['Calories_log']
X_test = test[features]
X_scaled=sc.fit_transform(train[features])
X_test_scaled=sc.transform(test[features])

In [10]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error

In [11]:
xgb_model=XGBRegressor(max_depth=6,n_estimators=1000,learning_rate=0.05,random_state=42)

In [12]:
xgb_model.fit(X_scaled,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [13]:
y_pred_log = xgb_model.predict(X_test_scaled)
y_pred = np.expm1(y_pred_log)

In [14]:
submission=pd.DataFrame({'id':test['id'] ,'Calories':y_pred})
submission.to_csv('/kaggle/working/submission.csv',index=False)

In [15]:
y_pred_train_log = xgb_model.predict(X_scaled)
y_pred_train = np.expm1(y_pred_train_log)
rmsle = np.sqrt(mean_squared_log_error(np.expm1(y), y_pred_train))
print(f'RMSLE: {rmsle:.5f}')

RMSLE: 0.05577
